In [ ]:
#!pip install tensorflow==2.15.0  # Erstat med den ønskede version. Denne er nødvendig for at bruge TCN
#!pip install keras_tuner
#!pip install keras-tcn
# Grundlæggende pakker
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning med TensorFlow og Keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model

import logging
import os
import sys
import importlib

# Check if GPU is available and configure memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and memory growth enabled")
    except RuntimeError as e:
        print("Memory growth setting failed:", e)
else:
    print("No GPU found. Using CPU.")

# Check TensorFlow version for reference
print("TensorFlow Version:", tf.__version__)

# Additional GPU details
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", gpus)

# Optional: To specify a particular GPU, if multiple are available
gpu_number = 0  # Change this index if needed
if len(gpus) >= 2:
    try:
        tf.config.experimental.set_visible_devices(gpus[gpu_number], 'GPU')
        print(f"Using GPU: {gpus[gpu_number]}")
    except RuntimeError as e:
        print("Failed to set specific GPU:", e)

ROOT_PATH = '/Users/rasmusklitteandersen/Library/CloudStorage/GoogleDrive-rasmusklitteandersen@gmail.com/Mit drev/speciale/'

# ROOT_PATH = '/content/drive/MyDrive/speciale/'
# from google.colab import drive
# drive.mount('/content/drive')

# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#   print('Not using a high-RAM runtime')
# else:
#   print('You are using a high-RAM runtime!')

In [ ]:
# Set up paths and configurations
#ROOT_PATH = '/content/drive/MyDrive/speciale/'
sys.path.append(ROOT_PATH)
os.chdir(ROOT_PATH)

# Import custom utilities and modules
from utils.misc import LoadData, TerminateNaN, Plotting, EvaluationMetric
from utils.models import ModelTrainer

# ===== CONFIGURATION SECTION =====
# Paths
DATA_PATH = f'{ROOT_PATH}data/final_dataset_test.csv'
MODEL_PATH = f'{ROOT_PATH}models/'
IMAGE_PATH = f'{ROOT_PATH}/images/'
LOG_DIR = f'{MODEL_PATH}/logs/'
TUNING_DIR = f'{MODEL_PATH}/tuning/'
RESULTS_DIR = f'{ROOT_PATH}results/'
TABLES_DIR = f'{ROOT_PATH}tables/'

# Flags and Options
INCLUDE_LAGS = True
INCLUDE_SEASON_VARS = True
INCLUDE_WEATHER = True
TEST = False
TIME_START = '2019-10-31'
TIME_END_PERIODS = ['2021-09-30', '2023-01-01', '2024-07-01']
MODELS = ['LSTM', 'TCN', 'Hybrid', 'Transformer']
#MODELS = ['Hybrid', 'Transformer']
TUNER = 'Hyperband'

# Training Parameters
TRAINING_EPOCH = 30
FINAL_MODEL_EPOCH = 50
BATCH_SIZE = 64
LOSS = 'mean_absolute_error'

# Generate dynamic strings based on flags
def get_extra_info():
    extra_info = ''
    if not INCLUDE_WEATHER:
        extra_info += '_no_weather'
    if not INCLUDE_LAGS:
        extra_info += '_no_lags'
    if not INCLUDE_SEASON_VARS:
        extra_info += '_no_season_vars'
    return extra_info

# ===== LOGGING SETUP =====
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# ===== FUNCTIONS =====

def reload_utils_misc():
    """Function to reload utils.misc module if changes are made."""
    import utils.misc
    importlib.reload(utils.misc)


def setup_directories(test_mode, image_path, log_dir, tuning_dir, results_dir, tables_dir):
    """Set up directories based on whether TEST mode is enabled."""
    test_suffix = 'test/'
    
    if test_mode: 
        image_path = f"{image_path}{test_suffix}"
        log_dir = f"{log_dir}{test_suffix}"
        tuning_dir = f"{tuning_dir}{test_suffix}"
        results_dir = f"{results_dir}{test_suffix}"
        tables_dir = f"{tables_dir}{test_suffix}"
        
    return image_path, log_dir, tuning_dir, results_dir, tables_dir


def initialize_data_loader():
    """Initialize the data loading utility with required flags."""
    return LoadData(INCLUDE_LAGS, INCLUDE_SEASON_VARS, INCLUDE_WEATHER, TUNER)

def initialize_evaluation_metric():
    """Initialize the evaluation metric utility with required flags."""
    return EvaluationMetric(INCLUDE_LAGS, INCLUDE_SEASON_VARS, INCLUDE_WEATHER, TUNER)

# ===== MAIN SCRIPT =====

if __name__ == "__main__":
    # Setup directories based on TEST mode
    # dirs = setup_directories(TEST)
    IMAGE_PATH, LOG_DIR, TUNING_DIR, RESULTS_DIR, TABLES_DIR = setup_directories(
    test_mode=TEST, 
    image_path=IMAGE_PATH, 
    log_dir=LOG_DIR, 
    tuning_dir=TUNING_DIR, 
    results_dir=RESULTS_DIR, 
    tables_dir=TABLES_DIR
    )

    extra_info = get_extra_info()
    
    # Reload utils if changes were made
    reload_utils_misc()

    # Initialize components
    load_data = initialize_data_loader()
    terminate_nan = TerminateNaN()
    evaluation_metric = initialize_evaluation_metric()

    # Set random seeds for reproducibility
    tf.random.set_seed(14)
    np.random.seed(14)

    # Log basic configuration
    logger.info(f"Starting training with configuration:")
    logger.info(f"Data path: {DATA_PATH}")
    logger.info(f"Model path: {MODEL_PATH}")
    logger.info(f"Model path: {TUNING_DIR}")
    logger.info(f"Include Lags: {INCLUDE_LAGS}, Include Season Vars: {INCLUDE_SEASON_VARS}, Include Weather: {INCLUDE_WEATHER}")
    logger.info(f"Training epochs: {TRAINING_EPOCH}, Final model epochs: {FINAL_MODEL_EPOCH}, Batch size: {BATCH_SIZE}")

# All models

## New setup

In [ ]:
#MODELS = ['LSTM','TCN', 'Hybrid', 'Transformer']

for i in range(len(MODELS)):
    MODEL = MODELS[i]
    print(f'Model: {MODEL}')
    for i in range(len(TIME_END_PERIODS)):
        
        # Set the seed for reproducibility
        tf.random.set_seed(14)
        np.random.seed(14)

        TIME_END = TIME_END_PERIODS[i]
        df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
        print(f'Time period: {TIME_PERIOD}')
        log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)
        datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)
        
        sequences_dict, scalers, X_test_scaled = load_data.prepare_sequences(df)
        datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)
        
        # Initialize and train the model
        trainer = ModelTrainer(datasets, log_dir, tuning_dir, LOSS, model_type=MODEL, sequences_dict=sequences_dict)
        final_model, history, best_hps, duration = trainer.train_model(BATCH_SIZE, TRAINING_EPOCH, FINAL_MODEL_EPOCH, overwrite=False)
        
        # Predictions and evaluation
        predictions = evaluation_metric.predict_full_sequence(final_model, X_test_scaled, scaler_y)
        if len(predictions) > len(y_test):
            predictions = predictions[:len(y_test)]
        else:
            y_test = y_test[:len(predictions)]
            
        evaluation_metric.evaluate_and_log(y_test, predictions, TIME_PERIOD, RESULTS_DIR, best_hps, duration, MODEL)
        
        # Plotting results
        plotting = Plotting(MODEL, TIME_PERIOD, IMAGE_PATH, INCLUDE_LAGS, INCLUDE_SEASON_VARS, INCLUDE_WEATHER, TUNER)

        plotting.plot_predictions(df, y, X_train, X_test, np.array(predictions[:len(y_test)]))
        plotting.plot_training_history(history)
        plot_model(final_model, to_file=f'{IMAGE_PATH}/{MODEL}/{MODEL}_model{extra_info}_{TIME_PERIOD}.png', show_shapes=True, show_layer_names=True)

## Single model

In [ ]:
MODEL = 'Transformer'
print(f'Model: {MODEL}')

for i in range(len(TIME_END_PERIODS)):
    # Set the seed for reproducibility
    tf.random.set_seed(14)
    np.random.seed(14)

    TIME_END = TIME_END_PERIODS[i]
    df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
    print(f'Time period: {TIME_PERIOD}')
    log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)
    sequences_dict, scalers, X_test_scaled = load_data.prepare_sequences(df)
    datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)
    
    # Initialize and train the model
    trainer = ModelTrainer(datasets, log_dir, tuning_dir, LOSS, model_type=MODEL, sequences_dict=sequences_dict)
    final_model, history, best_hps, duration = trainer.train_model(BATCH_SIZE, training_epoch=TRAINING_EPOCH, 
                                                                   final_model_epoch=FINAL_MODEL_EPOCH, overwrite=False)
    
    # Predictions and evaluation
    predictions = evaluation_metric.predict_full_sequence(final_model, X_test_scaled, scaler_y)
    if len(predictions) > len(y_test):
        predictions = predictions[:len(y_test)]
    else:
        y_test = y_test[:len(predictions)]
        
    evaluation_metric.evaluate_and_log(y_test, predictions, TIME_PERIOD, RESULTS_DIR, best_hps, duration, MODEL)
    
    # Plotting results
    plotting = Plotting(MODEL, TIME_PERIOD, IMAGE_PATH, INCLUDE_LAGS, INCLUDE_SEASON_VARS, INCLUDE_WEATHER, TUNER)

    plotting.plot_predictions(df, y, X_train, X_test, np.array(predictions[:len(y_test)]))
    plotting.plot_training_history(history)
    plot_model(final_model, to_file=f'{IMAGE_PATH}{MODEL}/{MODEL}_model{extra_info}_{TIME_PERIOD}.png', show_shapes=True, show_layer_names=True)

## SHAP Value

In [ ]:
TIME_END = TIME_END_PERIODS[2]
print(f'Time end: {TIME_END}')
MODEL = 'LSTM'
df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
print(f'Time period: {TIME_PERIOD}')
log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)

#datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)
sequences_dict, datasets, scalers, X_test_scaled, X = load_data.prepare_sequences(df)

# Initialize and train the model
trainer = ModelTrainer(datasets, log_dir, tuning_dir, LOSS, model_type=MODEL, sequences_dict=sequences_dict)
final_model, history, best_hps, duration = trainer.train_model(BATCH_SIZE, training_epoch=1, 
                                                               final_model_epoch=1, overwrite=False)


In [ ]:
import shap
import pandas as pd
# Assuming sequences_dict["test"]["X"][1] is intended to be converted into a DataFrame
# Assuming sequences_dict["test"]["X"][1] is intended to be converted into a DataFrame
X_test = sequences_dict["test"]["X"][1]

# Ensure X_test is a DataFrame
if isinstance(X_test, np.ndarray):
    # If it's a NumPy array, convert it into a DataFrame.
    X_test = pd.DataFrame(X_test, columns=[f"Feature {i}" for i in range(X_test.shape[1])])

# Function to reshape and predict using the final model
def model_predict(data):
    # Reshape the input for prediction (expected: num_samples, 1, 18)
    data_reshaped = data.reshape((-1, 1, X_test.shape[1]))  # Now using the correct number of features
    return final_model(data_reshaped).numpy()  # Make prediction

# Create the SHAP KernelExplainer object with the original data
explainer = shap.KernelExplainer(model_predict, X_test[:100])  # Using a small background dataset

# Reshape for predictions – keep it as (10, number_of_features)
reshaped_for_prediction = X_test[:10]  # Using the first 10 instances

# Calculate SHAP values for the first 10 instances
shap_values = explainer.shap_values(reshaped_for_prediction)  # Shape is (10, number_of_features)

# Visualize SHAP values for the first instance
shap.initjs()

# Manually create an Explanation object to include feature names
expl = shap.Explanation(
    values=shap_values[0],                  # SHAP values for the first instance
    feature_names=X_test.columns.tolist()    # Pass feature names from the DataFrame
)

# Create a bar plot to summarize SHAP values for the first instance
shap.plots.bar(expl)

In [ ]:
data = np.squeeze(sequences_dict["test"]["X"][:10])

data_reshaped = data.reshape((-1, 1, X_test.shape[1])) 
print(pd.DataFrame(data_reshaped).shape)

In [ ]:
import numpy as np
import pandas as pd
import shap

# Assuming sequences_dict["test"]["X"][1] should be a DataFrame, convert to DataFrame if necessary
X_test = sequences_dict["test"]["X"][:10]

# Ensure X_test is a DataFrame
if isinstance(X_test, np.ndarray):
    # If it's a NumPy array, you can create a DataFrame with dummy column names or your own naming.
    X_test = pd.DataFrame(X_test)

def model_predict(data):
    # Reshape the input for prediction (expected: num_samples, 1, 18)
    data_reshaped = np.squeeze(data).reshape((-1, 1, X_test.shape[1]))  # Reshape to (num_samples, 1, 18)
    return final_model(data_reshaped).numpy()  # Make prediction

# Create the SHAP KernelExplainer object with the original data
explainer = shap.KernelExplainer(model_predict, X_test[:100])  # Using the original data for background

# Reshape for predictions – Keep it as (10, 18)
reshaped_for_prediction = X_test[:10]  # Using the first 10 instances

# Calculate SHAP values for the first 10 instances
shap_values = explainer.shap_values(reshaped_for_prediction)  # Shape now (10, 18)

# Visualize SHAP values for the first instance
shap.initjs()

 # Manually create an Explanation object to include feature names
expl = shap.Explanation(
        values=shap_values[0],                  # SHAP values from the model
        feature_names=X_test.columns.tolist(),  # Pass feature names
        data=reshaped_for_prediction            # Original data for SHAP
    )

# Create a bar plot to summarize SHAP values for the first instance
shap.plots.bar(expl, max_display=len(X_test.columns))

In [ ]:
import numpy as np
import pandas as pd
import shap

# Create your augmented dataset from sequences_dict
X_test = sequences_dict["test"]["X"][:10]  # Get the first 100 samples from your augmented dataset
feature_names = sequences_dict["test"]["X"][1].columns

print(f"Original X_test type: {type(X_test)}, length: {len(X_test)}")  # Show the type and length of the list

# Ensure X_test is an array-like structure that can be processed
if isinstance(X_test, list):
    # Validate the structure of the input; print the first element's structure
    print(f"First element structure: {type(X_test[0])}, shape: {np.shape(X_test[0]) if isinstance(X_test[0], np.ndarray) else 'N/A'}")

    # Assuming X_test is a list of 3D arrays or lists
    # Check if nested list structure
    try:
        # Convert to NumPy array for easier manipulation
        X_test = np.array(X_test)  # Shape: (100, 48, 18)
    except Exception as e:
        print("Error converting X_test to array:", e)
        raise

if isinstance(X_test, np.ndarray):
    # Check the dimensions of the array
    if X_test.ndim == 3:
        # Averaging across the time steps to reduce to (100, 18)
        # Here, decide how you want to handle the time steps
        X_test = np.mean(X_test, axis=1)  # Taking the mean across the time steps
        # Convert into a DataFrame with features
        X_test = pd.DataFrame(X_test, columns=feature_names)
    elif X_test.ndim == 2:  # If it's of shape (100, 18)
        # Directly convert to DataFrame
        X_test = pd.DataFrame(X_test, columns=feature_names)

# Function to reshape and predict using the final model
def model_predict(data):
    # Reshape the input for prediction (expected: num_samples, 1, num_features)
    data_reshaped = data.reshape((-1, 1, X_test.shape[1]))  # Reshape to (num_samples, 1, num_features)
    return final_model(data_reshaped).numpy()  # Make prediction

# Create the SHAP KernelExplainer object with the modified dataset
explainer = shap.Explainer(model_predict, X_test.values)

# Use the entire dataset for computation
reshaped_for_prediction = X_test.values  # The modified dataset for SHAP values

# Calculate SHAP values for the modified dataset
shap_values = explainer.shap_values(reshaped_for_prediction)

# Visualize SHAP values for the first instance
shap.initjs()
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)


# Create a SHAP Explanation object for the first instance using actual column names
expl = shap.Explanation(
    values=mean_abs_shap_values
)

# Create a bar plot to summarize SHAP values for the first instance
shap.plots.bar(expl)



In [134]:
import numpy as np
import pandas as pd
import shap

# Create your augmented dataset from sequences_dict
X_test = sequences_dict["test"]["X"][:10]  # Get the first 10 samples

# Extract feature names
feature_names = sequences_dict["test"]["X"][1].columns.tolist()  # Ensure these are correct

# Ensure X_test is an array-like structure that can be processed
if isinstance(X_test, list):
    # Convert to NumPy array if it is a list of lists or arrays
    X_test = np.array(X_test)  # Shape should be (10, 48, 18)

if isinstance(X_test, np.ndarray):
    if X_test.ndim == 3:
        # Averaging across time steps to reduce to (10, 18)
        X_test = np.mean(X_test, axis=1)  # Shape will now be (10, 18)
    
    # Convert into a DataFrame with features
    X_test = pd.DataFrame(X_test, columns=feature_names[:X_test.shape[1]])  # Match to the number of columns

# Function to reshape and predict using the final model
def model_predict(data):
    # Reshape the input for prediction (expected: num_samples, 1, num_features)
    data_reshaped = data.reshape((-1, 1, X_test.shape[1]))  # Reshape to (num_samples, 1, num_features)
    return final_model(data_reshaped).numpy()  # Make prediction

def f(X):
    return final_model.predict([X[:, i] for i in range(X.shape[1])]).flatten()

# Create the SHAP Explainer object with the modified dataset
explainer = shap.Explainer(model_predict, X_test.values)

# Calculate SHAP values for the modified dataset
shap_values = explainer.shap_values(X_test.values)

# Visualize SHAP values for the first instance
shap.initjs()

# Compute mean absolute SHAP values across all instances
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# Create a SHAP Explanation object including the feature names
expl = shap.Explanation(
    values=mean_abs_shap_values,           # Mean absolute SHAP values
    feature_names=feature_names  # Pass feature names for proper labeling
)

# Create a bar plot to summarize SHAP values
shap.plots.bar(expl)

/opt/anaconda3/envs/speciale-env/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: keras_tensor_7. Received: the structure of inputs=('*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*')
  warnings.warn(


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32,), dtype=float32). Expected shape (None, 48, 18), but input has incompatible shape (32,)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtype=float32)', 'tf.Tensor(shape=(32,), dtyp

In [ ]:
#Visualize SHAP values for LSTM
shap.initjs()

# Create figure and axis
fig, ax = plt.subplots()

shap.plots.bar(shap_values_with_names,show=False, ax=ax, max_display=len(shap_values_with_names))
plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_bar_full_dataset.png', bbox_inches='tight')  # Save as PNG (you can choose other formats like PDF)
plt.close()
shap.plots.beeswarm(shap_values_with_names, max_display=len(shap_values_with_names), show=False)
plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_beeswarm_full_dataset.png', bbox_inches='tight')  # Save as PNG (you can choose other formats like PDF)
plt.close()
shap.plots.waterfall(shap_values_with_names[0],show=False)
plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_waterfall_full_dataset.png', bbox_inches='tight')  # Save as PNG (you can choose other formats like PDF)
plt.close()

# Force plot (for the first instance)
force_plot = shap.plots.force(shap_values_with_names[0])

# Save force plot as HTML
shap.save_html(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_force_plot_full_dataset.html', force_plot)

In [ ]:
MODEL = MODELS[1]
TIME_END = TIME_END_PERIODS[1]
df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
print(f'Time period: {TIME_PERIOD}')
log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)
datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)


print(len(X_test.columns))

In [ ]:
import shap

for i in range(len(MODELS)):
    MODEL = MODELS[i]
    print(f'Model: {MODEL}')
    for i in range(len(TIME_END_PERIODS)):
        # Set the seed for reproducibility
        tf.random.set_seed(14)
        np.random.seed(14)

        TIME_END = TIME_END_PERIODS[i]
        df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
        print(f'Time period: {TIME_PERIOD}')
        log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)
        datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)
        
        # Initialize and train the model
        trainer = ModelTrainer(datasets, log_dir, tuning_dir, LOSS, model_type=MODEL)
        final_model, history, best_hps, duration = trainer.train_model(BATCH_SIZE, TRAINING_EPOCH, FINAL_MODEL_EPOCH, overwrite=False)

        # SHAP values
        sample_size = 100
        X_sample = np.squeeze(datasets['X_train'][:sample_size])  #[:sample_size] #X_sample = X_train[:sample_size] #datasets['X_train'][:sample_size]
        X_test_sample = np.squeeze(datasets['X_test'][:sample_size])  #[:sample_size] #X_val_sample = X_test[:10] #datasets['X_val'][:sample_size]
        feature_names=X_test.columns

        explainer = shap.Explainer(final_model, X_sample)
        shap_values = explainer(X_test_sample)

        # Manually create an Explanation object to include feature names
        shap_values_with_names = shap.Explanation(
            values=shap_values,       # SHAP values from the model
            feature_names=feature_names, # Pass feature names
            data=X_test_sample          # Original data for SHAP
        )
        
        #Visualize SHAP values for LSTM
        shap.initjs()

        # Create figure and axis
        fig, ax = plt.subplots()

        shap.plots.bar(shap_values_with_names,show=False, ax=ax, max_display=len(shap_values_with_names))
        plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_bar_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.png', bbox_inches='tight')  # Save as PNG (you can choose other formats like PDF)
        plt.close()
        shap.plots.beeswarm(shap_values_with_names, max_display=len(shap_values_with_names), show=False)
        plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_beeswarm_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.png', bbox_inches='tight')  # Save as PNG (you can choose other formats like PDF)
        plt.close()
        shap.plots.waterfall(shap_values_with_names[0],show=False)
        plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_waterfall_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.png', bbox_inches='tight')  # Save as PNG (you can choose other formats like PDF)
        plt.close()

        # Force plot (for the first instance)
        force_plot = shap.plots.force(shap_values_with_names[0])

        # Save force plot as HTML
        shap.save_html(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_force_plot_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.html', force_plot)

In [ ]:
import shap
from tqdm import tqdm

# Wrap the outer loop with tqdm for model progress
for i in tqdm(range(len(MODELS)), desc="Processing Models"):
    MODEL = MODELS[i]
    print(f'Model: {MODEL}')
    
    # Wrap the inner loop with tqdm for time period progress
    for i in tqdm(range(len(TIME_END_PERIODS)), desc="Processing Time Periods", leave=False):
        # Set the seed for reproducibility
        tf.random.set_seed(14)
        np.random.seed(14)

        TIME_END = TIME_END_PERIODS[i]
        df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
        print(f'Time period: {TIME_PERIOD}')
        log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)
        datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)
        
        # Initialize and train the model
        trainer = ModelTrainer(datasets, log_dir, tuning_dir, LOSS, model_type=MODEL)
        final_model, history, best_hps, duration = trainer.train_model(BATCH_SIZE, TRAINING_EPOCH, FINAL_MODEL_EPOCH, overwrite=False)

        # SHAP values
        sample_size = 100
        X_sample = np.squeeze(datasets['X_train'][:sample_size])
        X_test_sample = np.squeeze(datasets['X_test'][:sample_size])
        feature_names = X_test.columns

        explainer = shap.Explainer(final_model, X_sample)
        shap_values = explainer(X_test_sample)

        # Manually create an Explanation object to include feature names
        shap_values_with_names = shap.Explanation(
            values=shap_values,       # SHAP values from the model
            feature_names=feature_names, # Pass feature names
            data=X_test_sample          # Original data for SHAP
        )
        
        # Visualize SHAP values for LSTM
        shap.initjs()

        # Create figure and axis
        fig, ax = plt.subplots()
        shap.plots.bar(shap_values_with_names, show=False, ax=ax, max_display=len(shap_values_with_names))
        plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_bar_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.png', bbox_inches='tight')
        plt.close()
        
        shap.plots.beeswarm(shap_values_with_names, max_display=len(shap_values_with_names), show=False)
        plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_beeswarm_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.png', bbox_inches='tight')
        plt.close()
        
        shap.plots.waterfall(shap_values_with_names[0], show=False)
        plt.savefig(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_waterfall_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.png', bbox_inches='tight')
        plt.close()

        # Force plot (for the first instance)
        force_plot = shap.plots.force(shap_values_with_names[0])

        # Save force plot as HTML
        shap.save_html(f'{IMAGE_PATH}{MODEL}/{MODEL}_shap_force_plot_{TIME_PERIOD}_{extra_info}_sample_{sample_size}.html', force_plot)


In [ ]:
#Visualize SHAP values for LSTM
shap.initjs()

# Create figure and axis
fig, ax = plt.subplots()

shap.plots.bar(shap_values_with_names, ax=ax, max_display=len(shap_values_with_names))
shap.plots.beeswarm(shap_values_with_names, max_display=len(shap_values_with_names))
shap.plots.waterfall(shap_values_with_names[0], max_display=len(shap_values_with_names))
shap.plots.force(shap_values_with_names[0])

# Diebold-Mariano

In [ ]:
import pickle


TIME_END = TIME_END_PERIODS[2]
print(f'Time end: {TIME_END}')
MODEL = 'Hybrid'
df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
print(f'Time period: {TIME_PERIOD}')

with open(f'/Users/rasmusklitteandersen/Library/CloudStorage/GoogleDrive-rasmusklitteandersen@gmail.com/Mit drev/speciale/results/test/predictions/{MODEL}/{MODEL}_{TIME_PERIOD}_Hyperband_predictions.pkl', 'rb') as f:
    predictions_1 = pickle.load(f)


MODEL = 'Transformer'
with open(f'/Users/rasmusklitteandersen/Library/CloudStorage/GoogleDrive-rasmusklitteandersen@gmail.com/Mit drev/speciale/results/test/predictions/{MODEL}/{MODEL}_{TIME_PERIOD}_Hyperband_predictions.pkl', 'rb') as f:
    predictions_2 = pickle.load(f)



log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)
datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import acf
from scipy import stats

#mae = metrics.mae(np.array(y_test[:len(predictions)]), np.array(predictions))

errors_model1 = np.abs(np.array(y_test[:len(predictions)]) - predictions_1)
errors_model2 = np.abs(np.array(y_test[:len(predictions)]) - predictions_2)
d = errors_model1 - errors_model2
print(d)

# Sample function for Diebold-Mariano test
def diebold_mariano_test(errors_model1, errors_model2):
    d = errors_model1 - errors_model2
    d = np.ravel(d)
    mean_d = np.mean(d)
    acf_values = acf(d, fft=True)
    variance_d = np.var(d) * (1 + 2 * sum(acf_values[1:])) / len(d)
    dm_stat = mean_d / np.sqrt(variance_d)
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    return dm_stat, p_value

# Perform Diebold-Mariano test on the errors
dm_stat, p_value = diebold_mariano_test(errors_model1, errors_model2)
print(f"Diebold-Mariano Test Statistic: {dm_stat}")
print(f"P-Value: {p_value}")

# Interpretation
if p_value < 0.05:
    print("The difference in forecasting errors is statistically significant.")
else:
    print("The difference in forecasting errors is not statistically significant.")



# Old

In [ ]:

for i in range(len(MODELS)):
    MODEL = MODELS[i]
    print(f'Model: {MODEL}')
    for i in range(len(TIME_END_PERIODS)):
        # Set the seed for reproducibility
        tf.random.set_seed(14)
        np.random.seed(14)

        TIME_END = TIME_END_PERIODS[i]
        df, TIME_PERIOD = load_data.load_and_preprocess_data(DATA_PATH, TIME_START, TIME_END)
        print(f'Time period: {TIME_PERIOD}')
        log_dir, tuning_dir = load_data.setup_directories(LOG_DIR, TUNING_DIR, TIME_PERIOD, MODEL)
        datasets, y, X_train, X_test, scaler_y, y_test = load_data.split_and_scale_data(df)
        
        # Initialize and train the model
        trainer = ModelTrainer(datasets, log_dir, tuning_dir, LOSS, model_type=MODEL)
        final_model, history, best_hps, duration = trainer.train_model(BATCH_SIZE, TRAINING_EPOCH, FINAL_MODEL_EPOCH, overwrite=False)
        
        # Predictions and evaluation
        predictions = evaluation_metric.predict_future_values(datasets, scaler_y, final_model, n_hours_ahead=24)
        evaluation_metric.evaluate_and_log(y_test, predictions, TIME_PERIOD, RESULTS_DIR, best_hps, duration, MODEL)
        
        # Plotting results
        plotting = Plotting(MODEL, TIME_PERIOD, IMAGE_PATH, INCLUDE_LAGS, INCLUDE_SEASON_VARS, INCLUDE_WEATHER, TUNER)
        plotting.plot_predictions(df, y, X_train, X_test, predictions)
        plotting.plot_training_history(history)
        plot_model(final_model, to_file=f'{IMAGE_PATH}/{MODEL}/{MODEL}_model{extra_info}_{TIME_PERIOD}.png', show_shapes=True, show_layer_names=True)